# NYC Traffic and Weather - Explainer Notebook
*For DTU course [http://www.kurser.dtu.dk/2013-2014/02806.aspx?menulanguage=en-GB](Social data analysis and visualization (02806)*

*Project Assignment B*

## Motivation
> What is your dataset?

Our dataset is [NYPD Motor Vehicle Collisions](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95). It contains records for every reported incident in the NYC area. Records are available from July 2012 till today. Specifically, where the collision took place, the cause of the collision, injuries, fatalities and more.

Thhe other dataset used was an extraction of weather conditions for the NYC area. This was pulled from [Weather Underground](https://www.wunderground.com) using their csv service for hourly updates. For each hour, there are measurements of the temperature, visibility, windspeed as well as the overall weather conditions such as Rain, Snow, Clear etc.

> Why did you choose this/these particular dataset(s)?

Choosing the collisions dataset, was mainly out of interest in finding out how and why collisions happen.

The weather dataset was an afterthough and was mainly something we wanted to investigate after having looked collisions and the road conditions / traffic infrastructure.

> What was your goal for the end user's experience?

The main goal is to provide the user/reader with knowledge about how the weather affects the road and in that sense the collision rate. We would assume that more collisions happens in bad conditions, but can we quantify that? The user should also end up with an idea of where incidents happens the most, maybe to give an idea for the government of NYC for improvements. 

## Basic Stats. Let's understand the dataset better
>Write about your choices in data cleaning and preprocessing

Talking about the collisions dataset, the cleaning was mainly done for the columns that we knew we wanted to investigate or otherwise we though was important in the data exploration. Notably we wanted to make sure that we had location data for all the rows we investigated.

The weather information was not directly available to us and required a lot of HTTP requests to Weather Undergrounds servers.

To avoid having to lookup in a secondary dataset, that is the weather information, we merged the two datasets together. For the most part we had weather data for each hour for all the rows we wanted to investigate, with only a combined gap of a couple of days. In additions some hours had more than one row of weather information.


>Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)
